# load training result

[http://cv-tricks.com/tensorflow-tutorial/save-restore-tensorflow-models-quick-complete-tutorial/](http://cv-tricks.com/tensorflow-tutorial/save-restore-tensorflow-models-quick-complete-tutorial/)

In [ ]:
import tensorflow as tf
from PIL import Image
import glob
%pylab inline

In [ ]:
IMAGE_HEIGHT = 200
IMAGE_WIDTH = 320
NUM_CHANNELS = 3
BATCH_SIZE = 10
labels =["mesa", "forest","desert","jungle", "eh"]

In [ ]:
checkpoint_file = "/notebooks/Minecraft-AI/mc-data/model_no_eh/model.ckpt"

In [ ]:
sess = tf.InteractiveSession()
sess.as_default()
saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
saver.restore(sess,checkpoint_file )#, tf.train.latest_checkpoint('/notebooks/Minecraft-AI/mc-data/model/'))
tf.global_variables_initializer().run()

In [ ]:
graph = tf.get_default_graph()

In [ ]:
test_data_node = graph.get_operation_by_name("test_data_node").outputs[0]
test_prediction = graph.get_operation_by_name("test_prediction").outputs[0]

In [ ]:
def scaleImg(img):
    img = np.array(Image.open(img)).astype(np.float32)
#     img = img.astype(np.float32)
    return (img-(255/2.0)) / 255

In [ ]:
def scaleImgs(img_dir):
    img_files = glob.glob(img_dir)[:BATCH_SIZE]
    data = np.array([scaleImg(img) for img in img_files])
    return data

In [ ]:
img_path= "/notebooks/Minecraft-AI/mc-data/forest_splited/006/*.jpg"
batch_data = scaleImgs(img_path)

In [ ]:
predictions = sess.run([test_prediction],
              feed_dict={test_data_node:batch_data})[0]
predictions = np.argmax(predictions, 1)

In [ ]:
maj = np.bincount(predictions).argmax() # get majority

In [ ]:
img_files = glob.glob(img_path)[:BATCH_SIZE]
print "majority: ", labels[maj]
for i, img in enumerate(img_files):
    plt.figure()
    plt.title("label: {}".format(labels[predictions[i]]))
    plt.imshow(Image.open(img))
    plt.plot()

In [ ]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [ ]:
sess.close()